Libraries

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

PROJECT_ROOT = Path("..").resolve()
DATA_DIR = PROJECT_ROOT / "data" / "Converted Images"
OUT_DIR = PROJECT_ROOT / "outputs"
SAMPLE_DIR = OUT_DIR / "samples" / "segmentation"
SAMPLE_DIR.mkdir(parents=True, exist_ok=True)

EXTS = {".jpg", ".jpeg", ".png", ".bmp"}
RANDOM_STATE = 42


Scan Dataset & Binary Labeling

In [2]:
def scan_dataset(data_dir: Path):
    class_dirs = sorted([p for p in data_dir.iterdir() if p.is_dir()])
    rows = []
    for cls_dir in class_dirs:
        cls = cls_dir.name
        y = 0 if cls.lower() == "healthy" else 1
        for fp in cls_dir.rglob("*"):
            if fp.is_file() and fp.suffix.lower() in EXTS:
                rows.append({"path": str(fp), "class": cls, "Output": y})
    return pd.DataFrame(rows)

df = scan_dataset(DATA_DIR)

print("Counts per folder:")
display(df["class"].value_counts())

print("\nBinary counts (Output):")
display(df["Output"].value_counts())

print("\nTotal:", len(df))


Counts per folder:


class
Healthy            136
Soft_Rot           129
Gray_Blight        119
Brown_Stem_Spot    119
Anthracnose        118
Stem_Canker        103
Name: count, dtype: int64


Binary counts (Output):


Output
1    588
0    136
Name: count, dtype: int64


Total: 724


K-Means Segmentation

In [ ]:
def kmeans_segment_labels(bgr, k=3):
    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
    h, w = lab.shape[:2]
    X = lab.reshape(-1, 3).astype(np.float32)

    km = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init="auto")
    labels = km.fit_predict(X).reshape(h, w)
    centers = km.cluster_centers_  
    return labels, centers

import numpy as np
import cv2

def pick_roi_cluster_smart(labels, centers):
    h, w = labels.shape
    k = centers.shape[0]

    L_vals = centers[:, 0]
    bg_by_light = int(np.argmax(L_vals))

    def border_touch_ratio(mask):
        border = np.zeros_like(mask, dtype=np.uint8)
        border[0, :] = 1; border[-1, :] = 1; border[:, 0] = 1; border[:, -1] = 1
        touch = int((mask & border).sum())
        area  = int(mask.sum()) + 1
        return touch / area

    def largest_component(mask):
        m = (mask * 255).astype(np.uint8)
        n, cc, stats, centroids = cv2.connectedComponentsWithStats(m, connectivity=8)
        if n <= 1:
            return 0, (w/2, h/2), mask
        areas = stats[1:, cv2.CC_STAT_AREA]
        idx = 1 + int(np.argmax(areas))
        area_l = int(stats[idx, cv2.CC_STAT_AREA])
        cx, cy = centroids[idx]
        keep = (cc == idx).astype(np.uint8)
        return area_l, (cx, cy), keep

    img_cx, img_cy = w/2.0, h/2.0
    best_k, best_score = 0, -1e18

    for kk in range(k):
        if kk == bg_by_light:
            continue  

        mask = (labels == kk).astype(np.uint8)
        area = int(mask.sum())
        if area < 0.05 * h * w:
            continue

        btr = border_touch_ratio(mask)
        area_l, (cx, cy), keep = largest_component(mask)

        compactness = area_l / (area + 1e-9)
        dist = np.sqrt((cx - img_cx)**2 + (cy - img_cy)**2)
        dist_norm = dist / (np.sqrt(img_cx**2 + img_cy**2) + 1e-9)

      
        score = (2.2 * compactness) - (2.0 * btr) - (0.8 * dist_norm)

        if score > best_score:
            best_score = score
            best_k = kk

    return best_k

def choose_background_cluster(labels):
    h, w = labels.shape
    k = int(labels.max()) + 1

    border = np.zeros_like(labels, dtype=bool)
    border[0, :] = True
    border[-1, :] = True
    border[:, 0] = True
    border[:, -1] = True

    scores = []
    for kk in range(k):
        area_frac = np.mean(labels == kk)
        border_frac = np.mean(labels[border] == kk)  
        score = 0.75 * border_frac + 0.25 * area_frac
        scores.append(score)

    return int(np.argmax(scores))

def keep_largest_component(mask):
    m = (mask * 255).astype(np.uint8)
    n, cc, stats, _ = cv2.connectedComponentsWithStats(m, connectivity=8)
    if n <= 1:
        return mask
    areas = stats[1:, cv2.CC_STAT_AREA]
    idx = 1 + int(np.argmax(areas))
    return (cc == idx).astype(np.uint8)


def refine_mask(mask01: np.ndarray) -> np.ndarray:
    """
    mask01: 0/1 uint8
    output: 0/1 uint8
    """
    mask = (mask01 > 0).astype(np.uint8)

    kernel = np.ones((7, 7), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN,  kernel, iterations=1)

    n, cc, stats, _ = cv2.connectedComponentsWithStats(mask * 255, connectivity=8)
    if n <= 1:
        return mask

    H, W = mask.shape
    best = None
    best_area = -1

    for i in range(1, n):
        x, y, w, h, area = stats[i]
        touches_border = (x == 0) or (y == 0) or (x + w == W) or (y + h == H)

        if (not touches_border) and area > best_area:
            best_area = area
            best = i

    if best is None:
        areas = stats[1:, cv2.CC_STAT_AREA]
        best = 1 + int(np.argmax(areas))

    return (cc == best).astype(np.uint8)


def make_roi_mask_from_labels(labels):
    bg_k = choose_background_cluster(labels)
    mask = (labels != bg_k).astype(np.uint8)  

    kernel = np.ones((7,7), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    mask = keep_largest_component(mask)

    return mask, bg_k


Visualization & Save Samples

In [4]:
def bgr_to_rgb(bgr): 
    return cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

def overlay_mask(bgr, mask):
    overlay = bgr.copy()
    red = np.zeros_like(bgr)
    red[:,:,2] = 255
    alpha = 0.35
    overlay[mask==1] = (alpha*red[mask==1] + (1-alpha)*overlay[mask==1]).astype(np.uint8)
    return overlay

sample_n = 12
sample_df = df.sample(min(sample_n, len(df)), random_state=RANDOM_STATE)

for i, r in enumerate(sample_df.to_dict("records"), start=1):
    bgr = cv2.imread(r["path"])
    if bgr is None:
        continue

    # Pipeline
    labels, centers = kmeans_segment_labels(bgr, k=3)
    roi_k = pick_roi_cluster_smart(labels, centers)
    mask, bg_k = make_roi_mask_from_labels(labels)
    mask = refine_mask(mask)
    over = overlay_mask(bgr, mask)



    label_vis = (labels.astype(np.float32) / labels.max() * 255).astype(np.uint8)
    label_vis = cv2.applyColorMap(label_vis, cv2.COLORMAP_JET)

    fig = plt.figure(figsize=(14, 4))
    ax1 = fig.add_subplot(1, 3, 1)
    ax2 = fig.add_subplot(1, 3, 2)
    ax3 = fig.add_subplot(1, 3, 3)

    ax1.imshow(bgr_to_rgb(bgr))
    ax1.set_title(f"Original - {r['class']}")
    ax1.axis("off")

    ax2.imshow(bgr_to_rgb(label_vis))
    ax2.set_title("KMeans (k=3) clusters")
    ax2.axis("off")

    ax3.imshow(bgr_to_rgb(over))
    ax3.set_title(f"ROI mask (cluster={roi_k})")
    ax3.axis("off")

    out_img = SAMPLE_DIR / f"seg_{i:02d}_{r['class']}.png"
    fig.savefig(out_img, bbox_inches="tight")
    plt.close(fig)

print("Saved segmentation samples to:", SAMPLE_DIR)


c:\Users\User\.conda\envs\pengenalan-pola\Lib\site-packages\threadpoolctl.py:1226: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


Saved segmentation samples to: E:\Kuliah\Pengenalan Pola\final-project\outputs\samples\segmentation
